<center><h1>Py-Galaxies examples</h1></center>

# SUB-HALO Stars

## Import modules

In [1]:
import PyGalaxies
import time
import numpy as np


model_param_filepath='Input_Params/input_params.yml'
debug_flag = False
verbosity = 1 
time_code = True

## Get model params and print

In [2]:
model_params = PyGalaxies.ModelParams(model_param_filepath,verbosity,debug_flag)
model_params.output_params() 

input_files         : {'graph_file': '../Input Data/mega_graph_mini_09-11.hdf5'}
output_files        : {'halo_file': '../Output Data/SMT13HaloOutput.hdf5', 'galaxy_file': '../Output Data/SMT13GalaxyOutput.hdf5'}
cosmology           : {'omega_m': {'Description': 'Matter density parameter', 'Value': 0.3, 'Units': 'None'}, 'f_baryon': {'Description': 'Baryon fraction', 'Value': 0.155, 'Units': 'None'}}
model_switches      : {'HOD': {'Description': 'Halo occupation description for stars (as primitive test)', 'Value': True}, 'sub_halo': {'Description': 'For each graph, load in Sub-halo data?', 'Value': True}}
performance         : {'io_nRec': {'Description': 'Size of HDF5 io buffer (number of records)', 'Value': 1000}}


## Run main routine

In [3]:
# Open the input HDF5 file containing graph groups and create an output file.
HDF_properties = PyGalaxies.HDFProperties(model_params)



start_time = time.time()


graph_arrays = []
graph_lengths = [] 
n_halo_in_graph = []


for graph_ID in range(0,HDF_properties.no_of_graphs)[-1:]:

    if verbosity >= 2: print('Processing graph',graph_ID)
    

    graph_properties = PyGalaxies.GraphProperties(graph_ID,
                                       HDF_properties.graph_input_file,
                                       model_params,
                                       HDF_properties.part_mass)

    
    array_of_halo_properties = np.empty(len(graph_properties.graph_halo_ids),
                                        dtype = object)
    
    if type(graph_properties.n_subhalos) == int:
        graph_lengths.append(np.nan)
        continue
    else:
        n_halo_in_graph.append(graph_properties.n_halos_in_graph)
        graph_lengths.append(np.sum(graph_properties.n_subhalos))
    
    for snap_ID in graph_properties.generation_id:
        

        if graph_properties.generation_start_index[snap_ID] == 2**30:
            continue
    
        
        this_generation_halo = graph_properties.graph_halo_ids[
                               graph_properties.generation_start_index[snap_ID]:
                               graph_properties.generation_start_index[snap_ID] + 
                               graph_properties.generation_length[snap_ID]]
        
        for halo_ID in this_generation_halo:
            
            halo = PyGalaxies.HaloProperties(str(graph_ID), snap_ID, halo_ID,
                                  graph_properties, HDF_properties.part_mass,
                                  HDF_properties.dtype_subhalo_stores)
        
        
            PyGalaxies.inclusive_mass_contribution(halo)
            
            if halo.nprog > 0: 
                PyGalaxies.gather_progenitors(halo,HDF_properties.part_mass)
            else:
                halo.mass_from_progenitors = 0.
            
            PyGalaxies.set_baryon_fraction(halo, array_of_halo_properties,
                                           model_params.f_baryon)
            
            halo.collect_galaxy_prog_info(graph_properties, HDF_properties.part_mass,
                                          array_of_halo_properties)
            
            halo.add_baryon_mass()
            halo.add_stellar_mass()
            
            halo.calculate_infall(model_params.f_baryon)
            
        
            halo.done=True
            
            array_of_halo_properties[halo_ID] = halo

            HDF_properties.n_halo +=1 

    # Save output
    HDF_properties.output_halos(array_of_halo_properties)

    graph_arrays.append(array_of_halo_properties)


# Close input file
HDF_properties.close_graph_io(HDF_properties.graph_input_file)


# Close output files
if HDF_properties.halo_output_iRec > 0: 

    halo_output_iRec = HDF_properties.flush_output()
    
HDF_properties.close_graph_io(HDF_properties.halo_output_file)


end_time = time.time()
print('This took {} seconds'.format(end_time-start_time))

13601526888.114677 [1.]
13480974197.478418 [1.36015269e+10]
13379832525.513159 [1.36015269e+10]
13305457401.025751 [1.36015269e+10]
13246836953.472397 [1.36015269e+10]
13199037566.435736 [1.36015269e+10]
13162335996.642332 [1.36015269e+10]
13138484535.915583 [1.36015269e+10]
13117875862.12014 [1.36015269e+10]
13102825667.289248 [1.36015269e+10]
This took 0.021600961685180664 seconds


In [4]:
vars(array_of_halo_properties[-1])

{'graph_ID': '975',
 'snap_ID': 61,
 'halo_ID': 22,
 'catalog_ID': 1816,
 'mass': 39903177160.09238,
 'nprog': 1,
 'prog_start': 21,
 'prog_end': 22,
 'prog_ids': array([21]),
 'prog_mass': array([3.99031772e+10]),
 'ndesc': -1,
 'desc_start': 1073741824,
 'desc_end': 1073741823,
 'desc_ids': array([], dtype=int32),
 'desc_mass': array([], dtype=float64),
 'mean_pos': array([33.91094678, 50.83798803, 36.65987155]),
 'velocity_dispersion_3D': 37.06783550241392,
 'mean_vel': array([-305.18989563,   60.86648401,  142.69911067]),
 'rms_radius': 0.044236223452654815,
 'redshift': 4.440892098500626e-16,
 'stellar_mass': array([1.36015269e+10]),
 'mass_baryon': array([1.28087554e+11]),
 'mass_from_progenitors': 5.307545158232393e+19,
 'mass_baryon_from_progenitors': 0.0,
 'inclusive_contribution': array([], dtype=float64),
 'done': True,
 'subhalo_start_index': 9,
 'n_subhalo': 1,
 'sub_halo_attrs': array([(9, 3.89219978e+19, 1.36015269e+10, False, 0.)],
       dtype=[('sub_graph_ids', '<i4')

In [38]:
for halo in array_of_halo_properties:
    try:
        print('{:.28e}'.format(halo.sub_halo_attrs['stellar_mass'][0]))
    except:
        print('hio')

hio
hio
hio
hio
hio
hio
hio
hio
hio
hio
hio
hio
hio
1.3601526888114677429199218750e+10
1.3601526889114677429199218750e+10
1.3601526890114677429199218750e+10
1.3601526891114677429199218750e+10
1.3601526892114677429199218750e+10
1.3601526893114677429199218750e+10
1.3601526894114677429199218750e+10
1.3601526895114677429199218750e+10
1.3601526896114677429199218750e+10
1.3601526897114677429199218750e+10


array([], dtype=float64)